# Machine Learning in Marketing by Python

## 1. Churn Predictions with Decision Tree

In [1]:
import pandas as pd

In [2]:
telco = pd.read_csv('telco.csv')
print(telco.head(5))

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [5]:
telco_raw = telco.copy()

# print the data types
print(telco_raw.dtypes)

# print the header
print(telco_raw.head(3))

# print unique values of each column
print(telco_raw.nunique())

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0

In [12]:
# Store numerical and categorical data

## store customerId and Churn column names
custid = ['customerID']
target = ['Churn']

## store categorical column names
categorical = telco_raw.nunique()[telco_raw.nunique()<5].keys().tolist()
categorical.remove(target[0])

## store numerical column names
numerical = [col for col in telco_raw.columns
                 if col not in custid + target + categorical]